<a href="https://colab.research.google.com/github/LikhanInSpace/Pripyat/blob/main/Resnet%20likhan%20version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# resize images 
img_size = [224,224]
train_path = '/content/drive/MyDrive/real vs fake/train'
valid_path = '/content/drive/MyDrive/real vs fake/test'

In [7]:
#Implementing ResNet built in function within Keras
#Pretrained on ImageNet weights
res = ResNet50(include_top=False, weights= 'imagenet',input_shape= img_size+ [3])

94773248/94765736 [==============================] - 1s 0us/step


In [8]:
for layer in res.layers:
  layer.trainable = False

In [9]:
folders = glob('/content/drive/MyDrive/real vs fake/train/*')

In [10]:
folders

['/content/drive/MyDrive/real vs fake/train/real',
 '/content/drive/MyDrive/real vs fake/train/fake']

In [11]:
len(folders)

2

In [12]:
x = Flatten()(res.output)

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=res.input, outputs=prediction)

In [14]:
#model.summary()

In [14]:
#assigning cost/loss function and optimization
#cost/loss function: categorical_crossentropy
#optimizer: adams
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [15]:
#importing images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [16]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/real vs fake/train',
                                                 target_size = (224, 224),
                                                 batch_size = 100,
                                                 class_mode = 'categorical')


Found 15024 images belonging to 2 classes.


In [17]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/real vs fake/test',
                                            target_size = (224, 224),
                                            batch_size = 100,
                                            class_mode = 'categorical')

Found 10000 images belonging to 2 classes.


In [18]:
#fit the model
r = model.fit(
    training_set, 
    validation_data = test_set,
    epochs = 15,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

Epoch 1/15
151/151 [==============================] - 7899s 52s/step - loss: 0.4537 - accuracy: 0.8757 - val_loss: 0.1534 - val_accuracy: 0.9430
Epoch 2/15
151/151 [==============================] - 209s 1s/step - loss: 0.1217 - accuracy: 0.9554 - val_loss: 0.1123 - val_accuracy: 0.9636
Epoch 3/15
151/151 [==============================] - 209s 1s/step - loss: 0.0883 - accuracy: 0.9691 - val_loss: 0.0548 - val_accuracy: 0.9790
Epoch 4/15
151/151 [==============================] - 209s 1s/step - loss: 0.0683 - accuracy: 0.9785 - val_loss: 0.0568 - val_accuracy: 0.9827
Epoch 5/15
151/151 [==============================] - 209s 1s/step - loss: 0.0698 - accuracy: 0.9763 - val_loss: 0.2184 - val_accuracy: 0.9091
Epoch 6/15
151/151 [==============================] - 209s 1s/step - loss: 0.0524 - accuracy: 0.9824 - val_loss: 0.0364 - val_accuracy: 0.9891
Epoch 7/15
151/151 [==============================] - 209s 1s/step - loss: 0.0452 - accuracy: 0.9856 - val_loss: 0.0649 - val_accuracy: 0.97

In [1]:
import matplotlib.pyplot as plt

In [2]:


# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: ignored